# Oficina 4 – Transações em Banco de Dados

Integrantes:
* André Luiz Girão Ferreira
* Sara
* Thaís
* Tiago

**Objetivo**: Experimentar estratégias para utilização de transações e níveis de isolamento em SGBDs relacionais. As tarefas envolvem uma simulação de um sistema de reservas de passagem áreas.








## Informações Iniciais - Simulação de Sistema de Reservas de Passagens

### Tabela e Atributos

Considere a seguinte tabela que registra os assentos reservados em um vôo:

***Assentos(num_voo,disp)***
onde

* ***num_voo***: é um número inteiro de 1 a 200
* ***disp***: é um atributo booleano cujo valor é **true** se o assento estiver vago e **false** caso contrário. **O valor inicial é *true***

### Reserva de Assento

A reserva de um assento é feita em três passos:

* **Passo 1:** O sistema recupera a lista dos assentos disponíveis.
* **Passo 2:** O cliente escolhe o assento. Esse passo deve ser simulado pela escolha aleatória de um dos assentos disponíveis, levando para isso um **tempo de escolha de 1 segundo**.
* **Passo 3:** O sistema registra a reserva do assento escolhido, **atualizando o valor de disp para false**.

Cada assento é reservado individualmente. Duas versões diferentes do processo de reserva devem ser implementadas:
* **Versão A:** A reserva é implementada como uma única transação que inclui os três passos acima.
* **Versão B:** A reserva inclui uma transação para o Passo 1 e outra para o Passo 3. O Passo 2 não faz parte das transações, mas deve ser executado.

### Os Agentes

Agentes de viagens são responsáveis por realizar as reservas de **200 clientes no total**. A atividade de um agente de viagens é simulada por uma ***thread***.

Experimentos devem ser realizados simulando a atuação de **k** agentes de viagem trabalhando simultaneamente, onde
* k = 1,2,4,6,8 e 10.

Cada agente/thread faz uma reserva de cada vez. **As threads devem ser reiniciadas até que todos os 200 clientes tenham seus assentos reservados**.

### Sobre os Experimentos

Dois conjuntos de experimentos devem ser feitos usando dois níveis de isolamento:  
* “read committed”; e
* “serializable”.  

Nos dois casos, o sistema deve ser configurado para **realizar bloqueios a nível de tupla** (linha).


### Preparação Inicial do Banco de Dados 

In [1]:
# instalando dependências
!pip install psycopg2

In [10]:
!pip uninstall psycopg2 psycopg2-binary
!pip install --no-cache-dir psycopg2-binary


In [59]:
# importando dependências
import threading
import random
import time
import psycopg2
from psycopg2 import errors # Para capturar erros específicos como deadlock


In [70]:
# --- Configurações para conectar ao banco de dados padrão 'postgres' (para criar dbs) ---
DB_CONFIG_ADMIN = {
    'host': 'localhost',
    'port': '5432',
    'dbname': 'postgres',  # Conectar ao banco de dados padrão 'postgres'
    'user': 'postgres',
    'password': '1234',
    'options': '-c client_encoding=UTF8'
}

# --- Configurações para conectar ao seu banco de dados 'oficina4' ---
DB_CONFIG_OFICINA4 = {
    'host': 'localhost',
    'port': '5432',
    'dbname': 'oficina4',
    'user': 'postgres',
    'password': '1234',
    'options': '-c client_encoding=UTF8'
}

TOTAL_CLIENTES = 200

In [ ]:
def get_conexao_db(options):
    return psycopg2.connect(**options)

In [51]:
def criar_banco_oficina4():
    """
    Cria o banco de dados 'oficina4' se ele ainda não existir.
    Conecta-se ao banco de dados administrativo ('postgres') para realizar esta operação.
    """
    conn_admin = None
    try:
        # Conexão inicial para criar o banco, usando as configurações administrativas
        conn_admin = get_conexao_db(DB_CONFIG_ADMIN)
        # Habilita o autocommit para comandos DDL (CREATE DATABASE)
        conn_admin.autocommit = True
        cur_admin = conn_admin.cursor()

        # Verifica se o banco de dados "oficina4" já existe
        cur_admin.execute("SELECT 1 FROM pg_database WHERE datname = 'oficina4'")
        existe = cur_admin.fetchone()
        if not existe:
            # Cria o banco de dados com codificação UTF8 e configurações de locale robustas
            cur_admin.execute("CREATE DATABASE oficina4 WITH ENCODING 'UTF8' LC_COLLATE 'C' LC_CTYPE 'C' TEMPLATE template0;")
            print("Banco de dados 'oficina4' criado com sucesso.")
        else:
            print("Banco de dados 'oficina4' já existe.")

        cur_admin.close()

    except psycopg2.Error as e:
        print(f"Erro ao criar o banco de dados 'oficina4': {e}")
        return False # Indica falha
    except Exception as e:
        print(f"Ocorreu um erro inesperado ao criar o banco: {e}")
        return False # Indica falha
    finally:
        if conn_admin is not None:
            conn_admin.close()
    return True # Indica sucesso

def criar_tabela_assentos():
    """
    Cria a tabela 'Assentos' no banco de dados 'oficina4' se ela ainda não existir.
    Conecta-se diretamente ao banco de dados 'oficina4' para realizar esta operação.
    """
    conn_oficina4 = None
    try:
        # Conecta ao banco "oficina4" para criar a tabela
        print("Tentando conectar ao banco de dados 'oficina4' para criar a tabela...")
        conn_oficina4 = get_conexao_db(DB_CONFIG_OFICINA4)
        # Habilita o autocommit para comandos DDL (CREATE TABLE)
        conn_oficina4.autocommit = True
        cur_oficina4 = conn_oficina4.cursor()

        # Cria a tabela Assentos se não existir
        cur_oficina4.execute("""
            CREATE TABLE IF NOT EXISTS Assentos (
                num_voo INTEGER PRIMARY KEY CHECK (num_voo BETWEEN 1 AND 200),
                disp BOOLEAN DEFAULT TRUE
            );
        """)
        print("Tabela 'Assentos' criada com sucesso.")

        cur_oficina4.close()

    except psycopg2.Error as e:
        print(f"Erro ao criar a tabela 'Assentos': {e}")
        return False # Indica falha
    except Exception as e:
        print(f"Ocorreu um erro inesperado ao criar a tabela: {e}")
        return False # Indica falha
    finally:
        if conn_oficina4 is not None:
            conn_oficina4.close()
    return True # Indica sucesso

In [62]:
# --- Inicializa os assentos (roda uma vez) ---
def inicializar_assentos():
    """
    Inicializa a tabela 'Assentos', limpando todos os dados existentes
    e inserindo 200 assentos, todos definidos como 'disp = TRUE'.
    Esta função deve ser rodada uma única vez para configurar a tabela.
    """
    conn = None
    cur = None
    try:
        conn = get_conexao_db(DB_CONFIG_OFICINA4)
        cur = conn.cursor()

        # Limpa a tabela antes de inicializar para garantir um estado limpo
        cur.execute("DELETE FROM Assentos;")
        print("Tabela 'Assentos' limpa para inicialização.")

        # Insere 200 assentos, todos como disponíveis
        for i in range(1, 201):
            cur.execute("INSERT INTO Assentos (num_voo, disp) VALUES (%s, TRUE);", (i,))
        
        conn.commit() # Confirma todas as inserções
        print("200 assentos inicializados como TRUE com sucesso.")

    except psycopg2.Error as e:
        print(f"Erro ao inicializar assentos: {e}")
        if conn:
            conn.rollback() # Garante rollback em caso de erro
    except Exception as e:
        print(f"Ocorreu um erro inesperado ao inicializar assentos: {e}")
        if conn:
            conn.rollback()
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

def limpar_assentos():
    """
    Define o status de 'disp' para TRUE para todos os assentos na tabela 'Assentos'.
    Útil para resetar o estado dos assentos entre testes.
    """
    conn = None
    cur = None
    try:
        conn = get_conexao_db(DB_CONFIG_OFICINA4)
        cur = conn.cursor()

        # Atualiza todos os assentos para disponível (TRUE)
        cur.execute("UPDATE Assentos SET disp = TRUE;")
        
        conn.commit() # Confirma a atualização
        print("Todos os assentos foram limpos (definidos como TRUE).")

    except psycopg2.Error as e:
        print(f"Erro ao limpar assentos: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"Ocorreu um erro inesperado ao limpar assentos: {e}")
        if conn:
            conn.rollback()
    finally:
        if cur:
            cur.close()
        if conn:
            conn.close()

In [63]:
criar_banco_oficina4()

Banco de dados 'oficina4' já existe.


True

In [64]:
criar_tabela_assentos()

Tentando conectar ao banco de dados 'oficina4' para criar a tabela...
Tabela 'Assentos' criada com sucesso.


True

In [65]:
inicializar_assentos()

Tabela 'Assentos' limpa para inicialização.
200 assentos inicializados como TRUE com sucesso.


### Tarefa 1

Implemente as **versões A e B do processo de reserva**. É importante que as implementações tratem adequadamente conflitos de Concorrência como **deadlocks e rollbacks**


In [73]:
# --- Versão A: tudo em uma transação ---
def reservar_assento_versao_a(id_agente, stop_event):
    """
    Tenta reservar um assento em uma única transação, usando FOR UPDATE para bloqueio.
    Continua tentando até que o evento de parada seja sinalizado.
    """
    while not stop_event.is_set(): # Loop para continuar tentando reservar
        conn = None
        cur = None
        try:
            conn = get_conexao_db(DB_CONFIG_OFICINA4)
            cur = conn.cursor()
            conn.autocommit = False # Inicia uma transação manual

            # Passo 1: Buscar assentos disponíveis E BLOQUEAR para UPDATE
            # ORDER BY num_voo para uma ordem previsível, pode ajudar a reduzir deadlocks em alguns casos
            cur.execute("SELECT num_voo FROM Assentos WHERE disp = TRUE ORDER BY num_voo ASC FOR UPDATE;")
            disponiveis = cur.fetchall()

            if not disponiveis:
                print(f"[Agente-{id_agente}]: Nenhum assento disponível. Sinalizando parada.")
                conn.rollback() # Libera qualquer lock residual
                stop_event.set() # Sinaliza para outras threads pararem
                break # Sai do loop do agente

            # Passo 2: Escolher assento (lógica fora do BD, mas com assentos já bloqueados)
            # Duração da reserva de 1 segundo aqui
            time.sleep(1) 
            escolhido = random.choice(disponiveis)[0]

            # Passo 3: Reservar o assento
            cur.execute("UPDATE Assentos SET disp = FALSE WHERE num_voo = %s;", (escolhido,))
            
            # Garante que a atualização realmente afetou uma linha.
            if cur.rowcount == 0:
                print(f"[Agente-{id_agente}]: Assento {escolhido} não foi atualizado (já reservado por outro?). Retentando...")
                conn.rollback() # Rollback se a atualização não ocorrer como esperado
            else:
                conn.commit() # Confirma a transação
                print(f"[Agente-{id_agente}]: Reservado assento {escolhido}")

        except errors.DeadlockDetected as e:
            print(f"[Agente-{id_agente}]: Deadlock detectado! Rollback e retentando. Erro: {e}")
            if conn:
                conn.rollback() # Essencial para desfazer a transação e liberar locks

        except psycopg2.Error as e:
            # Captura outros erros específicos do psycopg2 (ex: conexão perdida)
            print(f"[Agente-{id_agente}]: Erro no DB: {e}. Rollback e retentando...")
            if conn:
                conn.rollback()

        except Exception as e:
            # Captura erros inesperados
            print(f"[Agente-{id_agente}]: Erro inesperado: {e}. Sinalizando parada.")
            if conn:
                conn.rollback()
            stop_event.set() # Sinaliza para parar em caso de erro grave
            break # Sai do loop do agente
        finally:
            if cur:
                cur.close()
            if conn:
                conn.close()
        time.sleep(0.01) # Pequeno delay para evitar sobrecarga excessiva da CPU

# --- Versão B: duas transações ---
def reservar_assento_versao_b(id_agente, stop_event):
    """
    Tenta reservar um assento em duas transações separadas (seleção e atualização).
    Usa um UPDATE condicional para garantir atomicidade na reserva.
    Continua tentando até que o evento de parada seja sinalizado.
    """
    while not stop_event.is_set(): # Loop para continuar tentando reservar
        conn = None
        cur1 = None
        cur2 = None
        try:
            conn = get_conexao_db(DB_CONFIG_OFICINA4)
            conn.autocommit = False # Inicia uma transação manual

            # Transação 1: buscar assentos disponíveis (sem bloqueio)
            cur1 = conn.cursor()
            cur1.execute("SELECT num_voo FROM Assentos WHERE disp = TRUE ORDER BY num_voo ASC;")
            disponiveis = cur1.fetchall()

            if not disponiveis:
                print(f"[Agente-{id_agente}]: Nenhum assento disponível. Sinalizando parada.")
                conn.rollback() # Garante que a transação 1 seja desfeita (se houve alguma alteração)
                stop_event.set() # Sinaliza para outras threads pararem
                break # Sai do loop do agente

            conn.commit() # Fecha a transação 1, liberando o cursor1
            cur1.close()
            
            # Passo 2: Escolher assento (tempo de escolha, fora do DB)
            # Duração da reserva de 1 segundo aqui
            time.sleep(1) 
            escolhido = random.choice(disponiveis)[0]

            # Transação 2: Tentativa de reserva (usa o estado atual do DB)
            cur2 = conn.cursor()
            # Tenta atualizar APENAS se o assento ainda estiver disponível (disp = TRUE)
            cur2.execute("UPDATE Assentos SET disp = FALSE WHERE num_voo = %s AND disp = TRUE;", (escolhido,))
            
            if cur2.rowcount == 0:
                print(f"[Agente-{id_agente}]: Assento {escolhido} já foi reservado por outro agente ou não existe mais. Retentando...")
                conn.rollback() # Não conseguiu reservar, faz rollback da transação 2
            else:
                conn.commit() # Confirma a transação 2
                print(f"[Agente-{id_agente}]: Reservado assento {escolhido}")

        except errors.DeadlockDetected as e:
            print(f"[Agente-{id_agente}]: Deadlock detectado! Rollback e retentando. Erro: {e}")
            if conn:
                conn.rollback() # Essencial para desfazer a transação e liberar locks

        except psycopg2.Error as e:
            # Captura outros erros específicos do psycopg2
            print(f"[Agente-{id_agente}]: Erro no DB: {e}. Rollback e retentando...")
            if conn:
                conn.rollback()

        except Exception as e:
            # Captura outros erros inesperados
            print(f"[Agente-{id_agente}]: Erro inesperado: {e}. Sinalizando parada.")
            if conn:
                conn.rollback()
            stop_event.set() # Sinaliza para parar em caso de erro grave
            break # Sai do loop do agente
        finally:
            if 'cur1' in locals() and cur1 and not cur1.closed: cur1.close()
            if 'cur2' in locals() and cur2 and not cur2.closed: cur2.close()
            if conn:
                conn.close()
        time.sleep(0.01) # Pequeno delay para evitar sobrecarga excessiva da CPU

# --- Gerenciador de threads ---
def executar_reservas(versao, num_agentes):
    """
    Cria e gerencia threads de agentes para reservar assentos até que não haja mais.
    """
    print(f"\n--- Iniciando reservas versão {versao} com {num_agentes} agentes ---")
    agentes = []
    # Um evento para sinalizar que as threads devem parar
    stop_event = threading.Event() 
    
    reservar_assento_func = reservar_assento_versao_a if versao == "A" else reservar_assento_versao_b

    # Criando threads
    for i in range(num_agentes):
        id_agente = i + 1
        t = threading.Thread(target=reservar_assento_func, args=(id_agente, stop_event))
        agentes.append(t)
        t.start()

    # Esperando threads acabarem
    for agente in agentes:
        agente.join()

    print(f"--- Reservas versão {versao} finalizadas ---")

# --- Funções auxiliares para configuração do banco (assumindo que já existem) ---
# Você precisará ter essas funções definidas em outro lugar do seu código
# def criar_banco_oficina4(): ...
# def criar_tabela_assentos(): ...
# def inicializar_assentos(): ... (limpa a tabela e insere 200 assentos)
# def limpar_assentos(): ... (seta todos os 'disp' para TRUE)

In [74]:
# --- Bloco Principal de Execução ---
print("Por favor, certifique-se de que o banco 'oficina4' e a tabela 'Assentos' estão configurados e populados.")

# Testando com a Versão A
print("\n--- Iniciando Testes da Versão A ---")
limpar_assentos() # Limpa assentos para cada teste
executar_reservas(versao="A", num_agentes=5) # Exemplo com 5 agentes

# Testando com a Versão B
print("\n--- Iniciando Testes da Versão B ---")
limpar_assentos() # Limpa assentos antes da próxima versão
executar_reservas(versao="B", num_agentes=5) # Exemplo com 5 agentes

Por favor, certifique-se de que o banco 'oficina4' e a tabela 'Assentos' estão configurados e populados.

--- Iniciando Testes da Versão A ---
Todos os assentos foram limpos (definidos como TRUE).

--- Iniciando reservas versão A com 5 agentes ---
[Agente-1]: Reservado assento 186
[Agente-2]: Reservado assento 107
[Agente-3]: Reservado assento 151
[Agente-5]: Reservado assento 38
[Agente-4]: Reservado assento 194
[Agente-1]: Reservado assento 178
[Agente-2]: Reservado assento 145
[Agente-3]: Reservado assento 135
[Agente-5]: Reservado assento 71
[Agente-4]: Reservado assento 32
[Agente-1]: Reservado assento 198
[Agente-2]: Reservado assento 50
[Agente-3]: Reservado assento 22
[Agente-5]: Reservado assento 169
[Agente-4]: Reservado assento 125
[Agente-1]: Reservado assento 199
[Agente-2]: Reservado assento 42
[Agente-3]: Reservado assento 155
[Agente-5]: Reservado assento 70
[Agente-4]: Reservado assento 142
[Agente-1]: Reservado assento 156
[Agente-2]: Reservado assento 134
[Agente-3]:

### Tarefa 2

Apresente gráficos de linha onde, para cada valor de k (número de agentes) no eixo x, temos no eixo y o tempo necessário para que todos os clientes efetuem suas reservas. Um gráfico diferente deve ser apresentado para cada par de versões da reserva e nível de isolamento.



### Tarefa 3

Apresente uma tabela com o número máximo, mínimo e médio de vezes que um cliente teve que tentar reservar um assento até conseguir, ou seja, o número de vezes que uma reserva teve que ser refeita. A tabela considera as variações de k, versão de reserva e nível de isolamento.


### Tarefa 4

Apresente uma análise dos resultados obtidos em cada versão de reserva e tipo de isolamento, explicando as diferenças entre resultados.


### Tarefa 5

Análise de Conflitos de Concorrência. Para cada experimento executado, registre o número de deadlocks e rollbacks detectados pelo sistema gerenciador de banco de dados.

**O que entregar:** Um resumo tabular com o número total de deadlocks e rollbacks para cada combinação de (versão de reserva, nível de isolamento, valor de k). Indicação de como os erros foram tratados no código (ex: tentativas de reexecução, logs etc.).

### Tarefa 6 - Avaliação de Variação na Ordem de Alocação de Assentos

Compare a ordem final dos assentos ocupados (i.e., a sequência de num_voo com disp = false) entre diferentes execuções de um mesmo cenário com concorrência (mesmo k, versão e nível de isolamento), para identificar variações causadas por condições de corrida.

**O que entregar:** Para cada combinação de parâmetros com k > 1, execute o experimento 3 vezes e apresente a variação na ordem dos assentos alocados; Discuta a relação dessa variação com o nível de isolamento e a estrutura transacional usada.



### Tarefa 7 - Demonstração de Anomalias de Concorrência em Diferentes Níveis de Isolamento

O objetivo desta tarefa é observar experimentalmente a ocorrência (ou não) de três tipos clássicos de anomalias em transações concorrentes, sob os níveis de isolamento **READ COMMITTED** e **SERIALIZABLE**. Para isso, implemente três experimentos distintos e controlados, cada um projetado para testar um fenômeno. Cada experimento deve ser executado duas vezes, uma com cada nível de isolamento, e a diferença de comportamento deve ser registrada.



## Experimentos

### Experimento A - Non-repeatable Read

**Descrição:** Uma transação lê um mesmo dado duas vezes, mas entre essas duas leituras, outra transação modifica o dado e realiza commit.

**Cenário:**
* T1 inicia e lê o valor de um assento específico.
* T2 inicia, atualiza o mesmo assento para reservado (disp=false) e comita.
* T1 tenta reler o mesmo assento.

**O que entregar:**
* Código das transações T1 e T2.
* Logs ou saídas indicando se T1 leu valores diferentes na primeira e segunda leitura.
* Análise: isso ocorreu em qual nível de isolamento?




### Experimento B - Phanton Read

**Descrição:** Uma transação executa a mesma consulta duas vezes e obtém conjuntos de resultados diferentes porque outra transação inseriu ou removeu dados que se encaixam no critério da consulta.

**Cenário:**
* T1 inicia e consulta todos os assentos vagos (disp=true).
* T2 inicia, insere um novo assento vago (disp=true) ou atualiza um assento para vago, e comita.
* T1 executa novamente a mesma consulta.


**O que entregar:**
* Código de T1 e T2.
* Saída das duas execuções da consulta de T1.
* Indicação se houve alteração no resultado.
* Discussão da relação com o nível de isolamento usado.


### Experimento C - Dirty Read

**Descrição:** Uma transação lê dados modificados por outra transação que ainda não fez commit (ou que foi revertida). Esse experimento serve para mostrar que o PostgreSQL não permite dirty reads nem mesmo em READ COMMITTED, então o comportamento esperado é que não ocorra a leitura suja.

**Cenário:**
* T1 inicia e atualiza o valor de um assento (disp=false), mas não comita.
* T2 inicia e tenta ler o mesmo assento.
* T1 faz rollback.


**O que entregar:**
* Código de T1 e T2 com sincronização apropriada para simular esse cenário.
* Comprovação de que T2 não teve acesso ao valor não confirmado.
* Discussão: por que isso ocorre mesmo em READ COMMITTED?